<a href="https://colab.research.google.com/github/anmol-singh7/GenAI-Exploration/blob/main/Pinecone_VectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install tiktoken
!pip install langchain-community
!pip install sentence-transformers
!pip install huggingface_hub

In [ ]:
!pip install langchain_pinecone

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import OpenAIEmbeddings
# from langchain_community.llms import OpenAI
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.chains import retrieval_qa
from langchain.prompts import prompt
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_community.llms import HuggingFaceHub
import os
import time

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [ ]:
os.environ["PINECONE_API_KEY"]=""

In [ ]:
#if os.environ.get("this the key whose value to be get from env","if the key is not present then this value is assigned")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY","")
PINECONE_API_ENV = os.environ.get("PINECONE_API_ENV","gcp-starter")

In [ ]:
!mkdir pdfs

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")

In [ ]:
data=loader.load()

In [ ]:
data[0].page_content

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)

In [ ]:
text_chunks = text_splitter.split_documents(data)

In [ ]:
text_chunks

In [ ]:
print(text_chunks[0].page_content)

In [ ]:
print(len(text_chunks))

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    # model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
embedding_dimensions=len(embedding.embed_query("How are you"))

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
index_name = 'testing'
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=embedding_dimensions,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

In [ ]:
index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embedding)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]


In [ ]:
# vector_store.add_documents(documents=documents, ids=uuids)

In [ ]:
# vector_store.delete(ids=uuids[:-1])

In [ ]:
docsearch = vector_store.from_texts(
    texts=[d.page_content for d in text_chunks],
    embedding=embedding,
    index_name=index_name
    # metadatas=[d.metadata for d in text_chunks],
    # ids=[str(uuid4()) for _ in range(len(text_chunks))]
    )

In [ ]:
query = "YOLOv7 outperforms which model"
docs = docsearch.similarity_search(query)

In [ ]:
docs

In [ ]:
model_name ="google/byt5-small"
model_id = "google/flan-t5-large"
llm = HuggingFaceHub(
    repo_id=model_id,
    model_kwargs={"temperature":0}
)

In [ ]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
query = "YOLOv7 outperforms which models"

In [ ]:
qa.invoke(query)

{'query': 'YOLOv7 outperforms which models',
 'result': 'YOLOv5-X (r6.1), but improves AP by 2.'}

In [ ]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == "exit":
    print("Exiting")
    sys.exit()
  elif user_input == "":
    continue

  result = qa.invoke({"query":user_input})
  print(f"Answer: {result['result']}")

Input Prompt: who invented the yolo?
Answer: Answer: YOLOR-D6
Input Prompt: what was the accuracy of the yolov7?
Answer: 51.4%
Input Prompt: what was the accuracy of the yolov7?
Answer: 51.4%
Input Prompt: exit
Exiting


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
